# Imports and Setup

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from scipy import ndimage
import pickle
import copy
import random
import time

torch.set_printoptions(precision=3)
cuda = True if torch.cuda.is_available() else False
datasetCifar = False
amensiac = True
#[0, 1, len(dataLoad) * 0.1, len(dataLoad) * 0.2, len(dataLoad) * 0.3, len(dataLoad) * 0.4, len(dataLoad) * (0.5), len(dataLoad) * (0.6), len(dataLoad) * (0.7), len(dataLoad) * (0.8), len(dataLoad) * (0.9)]
idx = 0

# Data Entry and Processing

In [2]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,),(0.5)),
                                ])

In [3]:
# Using CIFAR100
traindata = datasets.CIFAR100('./data', download=True, train=True, transform=transform)
testdata = datasets.CIFAR100('./data', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Loaders that give 64 example batches
cifar_train_loader = torch.utils.data.DataLoader(traindata, batch_size=50, shuffle=True)
cifar_test_loader = torch.utils.data.DataLoader(testdata, batch_size=50, shuffle=True)

In [5]:
# Transform image to tensor and normalize features from [0,255] to [0,1]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,)),
                                ])

In [6]:
# Using MNIST
traindata = datasets.MNIST('./data', download=True, train=True, transform=transform)
testdata = datasets.MNIST('./data', download=True, train=False, transform=transform)

In [7]:
mnist_train_loader = torch.utils.data.DataLoader(traindata, batch_size=60, shuffle=True)
mnist_test_loader = torch.utils.data.DataLoader(testdata, batch_size=60, shuffle=True)

# Model

In [8]:
# Hyperparameters
log_interval = 10
num_classes = 100
torch.backends.cudnn.enabled = True
criterion = F.nll_loss

In [9]:
# Training method that saves batch updates
def train(model, epoch, loader, returnable=False, batchStop=0):
  model.train()
  deltas = []
  if amensiac:
    for _ in range(50):
        delta = {}
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
              delta[param_tensor] = 0
        deltas.append(delta)
  before = {}
  for param_tensor in model.state_dict():
      if "weight" in param_tensor or "bias" in param_tensor:
          before[param_tensor] = model.state_dict()[param_tensor].clone()
  for batch_idx, (data, target) in enumerate(loader):
    if batch_idx <= len((loader)) - 1 - batchStop:
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      if batch_idx % 10 == 0 and batch_idx < 500 and amensiac:
        after = {}
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
            after[param_tensor] = model.state_dict()[param_tensor].clone()
        for key in before:
          deltas[batch_idx // 10][key] = after[key] - before[key]
        for param_tensor in model.state_dict():
          if "weight" in param_tensor or "bias" in param_tensor:
            before[param_tensor] = model.state_dict()[param_tensor].clone()
      if batch_idx % log_interval == 0:
        print("\rEpoch: {} [{:6d}]\tLoss: {:.6f}".format(
            epoch, batch_idx*len(data),  loss.item()
        ), end="")
  return deltas

In [10]:
# Testing method
def test(model, loader, dname="Test set", printable=True):
  model.eval()
  test_loss = 0
  total = 0
  correct = 0
  with torch.no_grad():
    for data, target in loader:
      output = model(data)
      total += target.size()[0]
      test_loss += criterion(output, target).item()
      _, pred = torch.topk(output, 1, dim=1, largest=True, sorted=True)
      for i, t in enumerate(target):
        if t in pred[i]:
            correct += 1
  test_loss /= len(loader.dataset)
  if printable:
    print('{}: Mean loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        dname, test_loss, correct, total, 
        100. * correct / total
        ))
  return 1. * correct / total

# Original Training

In [11]:
trainingepochs = 10
num_classes = 100 if datasetCifar else 10
dataLoad = cifar_train_loader if datasetCifar else mnist_train_loader
dataTest = cifar_test_loader if datasetCifar else mnist_test_loader
stop = [0, 1, len(dataLoad) * 0.1, len(dataLoad) * 0.2, len(dataLoad) * 0.3, len(dataLoad) * 0.4, len(dataLoad) * (0.5), len(dataLoad) * (0.6), len(dataLoad) * (0.7), len(dataLoad) * (0.8), len(dataLoad) * (0.9)]
batchStop = stop[idx]

In [12]:
# load resnet 18 and change to fit problem dimensionality
resnet = models.resnet18()
resnet.conv1 = nn.Conv2d(3, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False) if datasetCifar else nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
resnet.fc = nn.Sequential(nn.Linear(512, num_classes), nn.LogSoftmax(dim=1))
optimizer = optim.Adam(resnet.parameters())

In [13]:
# Train new model for n epochs, saving parameter updates for
# sensitive batches

trainstart = time.process_time()
deltas = []
trainAcc = 0
if amensiac:  
  for _ in range(50):
    delta = {}
    for param_tensor in resnet.state_dict():
      if "weight" in param_tensor or "bias" in param_tensor:
          delta[param_tensor] = 0
    deltas.append(delta)
    
for epoch in range(1, trainingepochs+1):
  starttime = time.process_time()
  # train(resnet, epoch, all_data_train_loader, returnable=False)
  # batch = train(resnet, epoch, mnist_train_loader, returnable=True) 
  batch = train(resnet, epoch, dataLoad, returnable=True, batchStop=batchStop)
  if amensiac:
    for i in range(50):
      for key in deltas[i]:
          deltas[i][key] = batch[i][key] + deltas[i][key]
  # test(resnet, mnist_test_loader, dname="All data")
  trainAcc = test(resnet, dataTest, dname="All data")
  if epoch == trainingepochs:
    print("Accuracy: ", trainAcc)

  print(f"Time taken: {time.process_time() - starttime}")
endTimeTrain = time.process_time() - trainstart
print(f"Time taken for training: {endTimeTrain}")
accTime = {"accuracy": trainAcc, "time": endTimeTrain}


Epoch: 1 [ 59400]	Loss: 0.028251All data: Mean loss: 0.0011, Accuracy: 9818/10000 (98%)
Time taken: 1481.208765895
Epoch: 2 [ 59400]	Loss: 0.013988All data: Mean loss: 0.0011, Accuracy: 9825/10000 (98%)
Time taken: 1398.917518536
Epoch: 3 [ 59400]	Loss: 0.027693All data: Mean loss: 0.0006, Accuracy: 9895/10000 (99%)
Time taken: 1393.357703537
Epoch: 4 [ 59400]	Loss: 0.053666All data: Mean loss: 0.0005, Accuracy: 9902/10000 (99%)
Time taken: 1627.4425494860006
Epoch: 5 [ 59400]	Loss: 0.150741All data: Mean loss: 0.0006, Accuracy: 9891/10000 (99%)
Time taken: 1730.8849121350004
Epoch: 6 [ 59400]	Loss: 0.011948All data: Mean loss: 0.0009, Accuracy: 9845/10000 (98%)
Time taken: 1626.6741403329997
Epoch: 7 [ 59400]	Loss: 0.088650All data: Mean loss: 0.0006, Accuracy: 9897/10000 (99%)
Time taken: 1557.9982783119995
Epoch: 8 [ 59400]	Loss: 0.033886All data: Mean loss: 0.0006, Accuracy: 9908/10000 (99%)
Time taken: 1570.341156393999
Epoch: 9 [ 59400]	Loss: 0.031024All data: Mean loss: 0.0006, 

In [14]:
if batchStop != 0:
    accAndTime = F"CIFARCPUBatchStop" + str(batchStop) + ".pickle" if datasetCifar else F"MNISTCPUBatchStop" + str(batchStop) + ".pickle"
    with open(accAndTime, 'wb') as handle:
        pickle.dump(accTime, handle)

In [15]:
if batchStop != 0:
    with open(accAndTime, 'rb') as handle:
        b = pickle.load(handle)
    print(accAndTime)
    print(b)

In [16]:
deltaPickles = F"CIFARCPUdeltaPickles.pickle" if datasetCifar else F"MnistCPUdeltapickles.pickle"

with open(deltaPickles, 'wb') as handle:
    pickle.dump(deltas, handle)

In [17]:
path = F"./resnet/selective_mnist.pt"
torch.save({
            'model_state_dict': resnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, path)

In [18]:
path = F"resnet/selective_CIFAR.pt" if datasetCifar else F"resnet/selective_mnist.pt"
checkpoint = torch.load(path)
resnet.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
accuracy = [[],[],[],[],[],[],[],[],[],[]]
recordTime = [[],[],[],[],[],[],[],[],[],[]]
print(type(deltas))
# print(deltas)
# fileo = open(path, 'rb')
# deltas = pickle.load(fileo)
# fileo.close()
b = []
with open(deltaPickles, 'rb') as handle:
    b = pickle.load(handle)
print(len(b))
print(len(deltas))
import os
print(os.path.getsize(deltaPickles)/1073741824, ' GB')
# import sys
# print(sys.getsizeof(deltas)/1073741824)


<class 'list'>
50
50
2.0824923319742084  GB


In [20]:
# Begin amnesiac unlearning process, evaluating
# model accuracy as batches are removed
# 1 iteration = 1 batch
for j in range(10):
    random.shuffle(deltas)
    resnet.load_state_dict(checkpoint['model_state_dict'])
    for i in range(50):
        unlearnItStart = time.process_time()
        print(f"\riteration {j},{i}", end="")
        const = 1
        with torch.no_grad():
            state = resnet.state_dict()
            for param_tensor in state:
                if "weight" in param_tensor or "bias" in param_tensor:
                  state[param_tensor] = state[param_tensor] - const*deltas[i][param_tensor]
        resnet.load_state_dict(state)
        accuracy[j].append(test(resnet, dataTest, dname="All data"))
        end = time.process_time() - unlearnItStart
        recordTime[j].append(end)
        print(f"Time taken for unlearning: {end}")

iteration 0,0All data: Mean loss: 0.0009, Accuracy: 9854/10000 (99%)
Time taken for unlearning: 26.69598844900065
iteration 0,1All data: Mean loss: 0.0034, Accuracy: 9533/10000 (95%)
Time taken for unlearning: 26.123599061998902
iteration 0,2All data: Mean loss: 0.0039, Accuracy: 9378/10000 (94%)
Time taken for unlearning: 26.591601439000442
iteration 0,3All data: Mean loss: 0.0064, Accuracy: 9000/10000 (90%)
Time taken for unlearning: 25.938902605001203
iteration 0,4All data: Mean loss: 0.0118, Accuracy: 8548/10000 (85%)
Time taken for unlearning: 25.302068765000513
iteration 0,5All data: Mean loss: 0.0086, Accuracy: 8815/10000 (88%)
Time taken for unlearning: 26.375159473000167
iteration 0,6All data: Mean loss: 0.0190, Accuracy: 7627/10000 (76%)
Time taken for unlearning: 26.068253587000072
iteration 0,7All data: Mean loss: 0.0253, Accuracy: 7192/10000 (72%)
Time taken for unlearning: 26.396181785999943
iteration 0,8All data: Mean loss: 0.0290, Accuracy: 6545/10000 (65%)
Time taken f

KeyboardInterrupt: 

In [18]:
accuracyStore = F"CPUselective_acc_CIFAR.pk" if datasetCifar else F"CPUselective_acc_mnist.pk"
f = open(accuracyStore, "wb")
pickle.dump(accuracy, f)
f.close()

In [ ]:
timeStore = F"CPUselective_time_CIFAR.pk" if datasetCifar else F"CPUselective_time_mnist.pk"
f = open(timeStore, "wb")
pickle.dump(recordTime, f)
f.close()

In [19]:
accuracyStore = F"CPUselective_acc_CIFAR.pk" if datasetCifar else F"CPUselective_acc_mnist.pk"
timeStore = F"CPUselective_time_CIFAR.pk" if datasetCifar else F"CPUselective_time_mnist.pk"
res = []
with open(accuracyStore, 'rb') as handle:
    res = pickle.load(handle)
print(len(res))
letime = []
with open(timeStore, 'rb') as handle:
    letime = pickle.load(handle)
print(letime)